# Unit8 Part 2

Import required Libraries

In [1]:
import sqlite3
import pandas as pd

Create connection to SQLite DB file

In [17]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

if None is conn:
    print("Connect failed. Invalid DB!")

Check using the connection for the required tables

In [19]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the names of the tables
for table in tables:
    print(table[0])

# Close the cursor and connection
cursor.close()

Bookings
Facilities
Members


#### Q10:

Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [25]:
df = pd.read_sql_query('''
SELECT 
	f.name AS facility,
	SUM(
        CASE WHEN m.memid = 0 THEN b.slots * f.guestcost
        ELSE b.slots * f.membercost
        END
    ) AS total_revenue
FROM Bookings b
INNER JOIN Facilities f on f.facid = b.facid
INNER JOIN Members m on m.memid = b.memid
GROUP BY facility
HAVING SUM(
	CASE
    WHEN m.memid = 0 THEN b.slots * f.guestcost
     	ELSE b.slots * f.membercost
    END) < 1000;
''', conn)

df

,facility,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


#### Q11:
Produce a report of members and who recommended them in alphabetic surname,firstname order

In [23]:
df = pd.read_sql_query('''
SELECT 
    CONCAT_WS(', ', m.surname, m.firstname) AS member,
    CONCAT_WS(', ', r.surname, r.firstname) AS recommender
FROM Members m
INNER JOIN Members r
ON m.memid = r.recommendedby
ORDER BY r.surname
''', conn)

df

,member,recommender
0,"Stibbons, Ponder","Bader, Florence"
1,"Stibbons, Ponder","Baker, Anne"
2,"Farrell, Jemima","Baker, Timothy"
3,"Rownam, Tim","Boothe, Tim"
4,"Smith, Darren","Butters, Gerald"
5,"Baker, Timothy","Coplin, Joan"
6,"Smith, Tracy","Crumpet, Erica"
7,"Joplette, Janice","Dare, Nancy"
8,"Butters, Gerald","Genting, Matthew"
9,"Purview, Millicent","Hunt, John"


#### Q12:
Find the facilities with their usage by member, but not guests

In [26]:
df = pd.read_sql_query('''
SELECT mb.facid, mb.facility 
FROM (
  	SELECT DISTINCT f.name as facility, f.facid
	FROM Bookings b
	INNER JOIN Facilities f ON f.facid = b.facid
	WHERE b.memid > 0
) mb 
LEFT JOIN (	
	SELECT DISTINCT f.name as facility, f.facid
	FROM Bookings b
	INNER JOIN Facilities f ON f.facid = b.facid
	WHERE b.memid = 0
) gb
ON gb.facid = mb.facid
WHERE gb.facid IS NULL
ORDER BY mb.facility
    
''', conn)

df

,facid,facility


#### Q13:
Find the facilities usage by month, but not guests

In [31]:
df = pd.read_sql_query('''
SELECT 
    f.name AS facility_name,
    strftime('%Y-%m', b.starttime) AS month,
    COUNT(*) AS usage_count
FROM 
    Bookings b
JOIN 
    Facilities f ON b.facid = f.facid
WHERE 
    b.memid != 0
GROUP BY 
    f.name, strftime('%Y-%m', b.starttime)
ORDER BY 
    facility_name, month
''', conn)

df

,facility_name,month,usage_count
0,Badminton Court,2012-07,51
1,Badminton Court,2012-08,132
2,Badminton Court,2012-09,161
3,Massage Room 1,2012-07,77
4,Massage Room 1,2012-08,153
5,Massage Room 1,2012-09,191
6,Massage Room 2,2012-07,4
7,Massage Room 2,2012-08,9
8,Massage Room 2,2012-09,14
9,Pool Table,2012-07,103


###